# **Resgatar a ultima data de cada produto**

Muitas vezes precisamos pegar o maior valor por dia, e não existe um botão que faça isso mágicamente no PowerQuery, mas é possosivel fazer da seguinte forma:

1) Primeiro agrupa pelas colunas (Exemplo: Produto, Estado, Vendedor, etc) agregando por Todas as Linhas

2) Crie uma Coluna Personalizada chamada **Table.Max[Coluna_criada_etapa_anterior], "Coluna Data")**

3) Sera criado uma coluna, então remova as outras colunas e deixe apenas ela.

4) Expanda os valores da coluna, que nela exibira o resultado desejado

Abaixo Segue o código fonte

In [ ]:
let
    Fonte = f_Vendas_Camada_Silver,
    #"Linhas Agrupadas" = Table.Group(Fonte, {"produto", "Estado"}, {{"Tabela", each _, type table [produto=nullable text, Venda=nullable text, Data=nullable text, Estado=nullable text]}}),
    #"Personalização Adicionada" = Table.AddColumn(#"Linhas Agrupadas", "Última Venda", each Table.Max([Tabela],"Data")),
    #"Outras Colunas Removidas" = Table.SelectColumns(#"Personalização Adicionada",{"Última Venda"}),
    #"Última Venda Expandido" = Table.ExpandRecordColumn(#"Outras Colunas Removidas", "Última Venda", {"produto", "Venda", "Data", "Estado"}, {"produto", "Venda", "Data", "Estado"})
in
    #"Última Venda Expandido"

Nesse segundo exemplo é semelhante, mas agora é resgatar o maior valor no mês por produto

Utilizando a mesma tabela, foi criado
1) Colunas Mês/Ano (existe várias formas de criar, mas nesse exemplo criamos primeiro o mes, depois o ano, e logo mesclamos)
2) Agrupamos por Mes/Ano e Produto agregando todas as linhas
3) Criado uma Coluna Personalizada chamada **Table.Max[Coluna_criada_etapa_anterior], "Coluna Data")**

Veja o Código abaixo

In [ ]:
let
    Fonte = f_Vendas_Camada_Silver,
      #"Tipo Alterado" = Table.TransformColumnTypes(Fonte,{{"Data", type date}}),
    #"Nome do Mês Inserido" = Table.AddColumn(#"Tipo Alterado", "Nome do Mês", each Date.MonthName([Data]), type text),
    #"Ano Inserido" = Table.AddColumn(#"Nome do Mês Inserido", "Ano", each Date.Year([Data]), Int64.Type),
    #"Colunas Mescladas" = Table.CombineColumns(Table.TransformColumnTypes(#"Ano Inserido", {{"Ano", type text}}, "pt-BR"),{"Nome do Mês", "Ano"},Combiner.CombineTextByDelimiter("/", QuoteStyle.None),"Mes/Ano"),
    #"Linhas Agrupadas" = Table.Group(#"Colunas Mescladas", {"Mes/Ano", "produto"}, {{"Contagem", each _, type table [produto=nullable text, Venda=nullable text, Data=nullable date, Estado=nullable text, #"Mes/Ano"=text]}}),
    #"Personalização Adicionada" = Table.AddColumn(#"Linhas Agrupadas", "Personalizar", each Table.Max([Contagem],"Venda")),
    #"Outras Colunas Removidas" = Table.SelectColumns(#"Personalização Adicionada",{"Personalizar"}),
    #"Personalizar Expandido" = Table.ExpandRecordColumn(#"Outras Colunas Removidas", "Personalizar", {"produto", "Venda", "Data", "Estado", "Mes/Ano"}, {"produto", "Venda", "Data", "Estado", "Mes/Ano"})
in
    #"Personalizar Expandido"